In [1]:
import firefin
import pandas as pd
from grs import grs_test
from regression import RollingRegressor


In [2]:
#custumize
def customize_parameter(N_asset: int, T_time: int, windowsize: int, regression_until):
    return N_asset, T_time, windowsize, regression_until

N_asset, T_time, windowsize, regression_until = customize_parameter(
    50, 200, 60, "2009-10-26 15:00:00"
)

In [5]:
#get data
data = firefin.fetch_data(["return_adj", "cn_bond_30y"])
df2 = data["return_adj"]
thirty_year_bond=data["cn_bond_30y"]
#you can change the following scale of dataframe that align with your purpose
df3=df2.iloc[:T_time]
df2=df2.iloc[:T_time,:N_asset]


D:\Anaconda\envs\firefin\Lib\site-packages\firefin\data\gateway.py:85: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(obj)
D:\Anaconda\envs\firefin\Lib\site-packages\firefin\data\gateway.py:85: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(obj)
2025-05-12 21:20:46.264 | WARNING  | firefin.data.gateway:fetch_data:143 - cn_bond_30y is not a valid data name, mock with random data


stock_code           000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
datetime                                                                     
2009-01-05 15:00:00   0.026427   0.038760   0.036517   0.032258   0.050710   
2009-01-06 15:00:00   0.060762   0.029851   0.029810   0.031250   0.040541   
2009-01-07 15:00:00  -0.030097  -0.005797   0.050000  -0.011364  -0.007421   
2009-01-08 15:00:00  -0.039039   0.005831   0.050125  -0.026820  -0.018692   
2009-01-09 15:00:00   0.026042  -0.001449   0.038186   0.043307   0.009524   

stock_code           000007.SZ  000008.SZ  000009.SZ  000010.SZ  000011.SZ  \
datetime                                                                     
2009-01-05 15:00:00   0.046296   0.022624   0.037688   0.019380   0.005141   
2009-01-06 15:00:00   0.032448   0.030974   0.038741   0.020913   0.033248   
2009-01-07 15:00:00  -0.031429   0.012875  -0.006993   0.020484   0.007426   
2009-01-08 15:00:00   0.035398   0.025424  -0.032864   0.012774

In [ ]:
market_data = df2.fillna(0)
T=market_data.shape[0]
N=market_data.shape[1]
thirty_year_bond = thirty_year_bond.head(T)
thirty_year_bond['cn_bond_30y']=(1+thirty_year_bond['cn_bond_30y']/100)**(1/365)-1

# 对齐到 market_data 的每一个时间戳上：
thirty_year_bonds = thirty_year_bond['cn_bond_30y'].reindex(market_data.index, method='ffill')
# 然后按行相加：
stocks_excess_return = market_data.add(-thirty_year_bonds, axis=0)
#构造市场因子
market_excess_return=df3.mean(axis=1).add(-thirty_year_bonds, axis=0)
market_excess_return_matrix=pd.concat([market_excess_return]*N, axis=1)
market_excess_return_matrix.columns = market_data.columns

In [ ]:
reg = RollingRegressor(
    x=market_excess_return,
    y=stocks_excess_return,
    mode="single",
    fit_intercept=True
)
window_size=windowsize              #输入窗口大小
result=reg.fit(window_size)

In [ ]:
alpha=result.alpha
beta=result.beta
residual=result.residuals
grs_test(residual, alpha,regression_until, window_size, market_excess_return)